## Most of this is a giant mess.

What we learned from it, however, is that using numba for a jit in certain circumstances is not feasable with completely general code. ie: The numeric_ODE module as it stands, is supposed to take any ODE as an input and integrate it. However, because of scoping issues and the necessity to pass an anonymous function through a function call, numba was not able

In [ ]:
import numpy as np
import numba as nb  #uncomment for numba
import matplotlib.pyplot as plt
%matplotlib inline

def julia(c):
    #@np.vectorize  #comment for numba
    @nb.vectorize #uncomment for numba
    def j(z):
        for n in range(100):
            z = z**2 + c
            if abs(z) > 2:
                return n
        return 0
    return j

j = julia(0.345 + 0.45j)

@nb.jit  #uncomment for numba
def cplane(min=-1.5, max=1.5, points=3000):
    r = np.linspace(-1.5, 1.5, points)
    x, y = np.meshgrid(r,r)
    z = x + y * 1j
    return z

%time z = cplane()
%time jset = j(z)

plt.figure(1, (20,15))
plt.imshow(jset, cmap=plt.cm.bone)
plt.xticks([])
plt.yticks([])
plt.title("Julia Set : c = 0.345 + 0.45j")
plt.show()

CPU times: user 384 ms, sys: 288 ms, total: 672 ms
Wall time: 1.91 s
CPU times: user 1.77 s, sys: 28 ms, total: 1.8 s
Wall time: 3.6 s


In [1]:
import numpy as np
import numeric_ode

In [2]:
u_0 = (0,1)
a = 0
b = 10
delta_t = 0.0001
mass = 1
delta = 0.25
force = 0.4
omega = 1

In [3]:
n = int((b - a) / float(delta_t)) #Number of points in t-mesh
tvals = np.linspace(a, b, n)
u_prime = lambda u, t: np.array((u[1], 1 / mass * (-delta * u[1] + u[0] - (u[0]) ** 3 + force * np.cos(omega * t))))
%time u_p = numeric_ode.integrate(u_0, a, b, delta_t, u_prime, 'rk4') #With Numba

TypingError: Failed at nopython (nopython frontend)
Untyped global name 'euler_step': cannot determine Numba type of <class 'function'>
File "numeric_ode.py", line 189

This error may have been caused by the following argument(s):
- argument 4: cannot determine Numba type of <class 'function'>


In [3]:
n = int((b - a) / float(delta_t)) #Number of points in t-mesh
tvals = np.linspace(a, b, n)
u_prime = lambda u, t: np.array((u[1], 1 / mass * (-delta * u[1] + u[0] - (u[0]) ** 3 + force * np.cos(omega * t))))
%time u_p = numeric_ode.integrate(u_0, a, b, delta_t, u_prime, 'rk4') #Without Numba

CPU times: user 1.82 s, sys: 0 ns, total: 1.82 s
Wall time: 1.88 s
